## Preliminaries

Activate the project environment:

In [56]:
using InstantiateFromURL
activate_github("QuantEcon/QuantEconLectureAllPackages", tag = "v0.9.0") # activate the QuantEcon environment

("v0.9.0", "/home/jupyter/ECON628_local/MetricProject_test/.projects/QuantEconLectureAllPackages-v0.9.0")

In [57]:
using LinearAlgebra, Statistics, Compat, Parameters, Random

In [58]:
using Distributions

# Discrete choice

In this exercise I have to create a data set of students, schools and distances. The student must choose between the schools he have in his menu taking in account the distance. With the data in hands, the exercise demand to estimate the parameters of the data generator process assuming two different hypothesis about the distribution of the unobservable component of the utility: (1) $\epsilon$ is normal; (2) $\epsilon$ is T1EV. The true $\epsilon$ is normal, however, we do not have a formula for the expected utility when $\epsilon$ is normal as we have in the T1EV case. So, in the first case we have a source of error associated to the fact we have to approximate the expectation and in second case we will have a source of error associated to the fact that we are not using the correct distribution.

Let's set up the parameters

In [59]:
N=500; #number of students
numSch=5; #number of schools
seed = 0;
μ = 0;

### Generating the data
Main parameters:

In [60]:
Random.seed!(seed+1); #fixing the seed
δ    = randn(numSch,1) .+ μ; #generating the parameter deltas (mean utility of each school)

In [61]:
dist = abs.(randn(N,numSch)); #generating the observable distance
γ    = 1.2;
σ    = sqrt((pi^2)/6.0) #I am fixing the variance of the normal to be equal of the T1EV(0,1)

1.282549830161864

Let's draw the utilities

In [62]:
Random.seed!(seed+2); 
ϵ=σ*randn(N,numSch);

Computing the utilities in a matrix

In [63]:
util = repeat(δ',N,1) .- γ*dist .+ ϵ;

Let's introduce a variation in the menu

In [64]:
menuSize = zeros(Int64,N,1);
choice   = zeros(Int64,N,1);
menu     = zeros(Bool, N,numSch);

In [65]:
Random.seed!(seed+3);
for i=1:N
    #Number of schools in the menu
    menuSize[i]=rand(DiscreteUniform(1,numSch-1)) + 1; 
    # Generate a random integer uniformaly distributed between 1 and (numSch-1)
    # I add 1 to assure that they will have at least 2 options
    
    #Identity of schools in the menu
    temp=randperm(numSch); 
    # return a random permutation of integers between 1 and NumSch
    
    for j=1:numSch
        if any(temp[1:menuSize[i]].==j) 
            # any() returns equal 1 if any element of the array is non-zero
            # 1:menuSize[i] is the menu of individual i
            # Thus I will put one in the matrix menu when the i have the
            # option j available
            menu[i,j]=true;
        end
    end   
    
    #Choose the school that provides highest utility
    choice[i]=findall(util[i,:].== findmax(util[i,menu[i,:]])[1])[1]; # I am assuming no ties!
    # I am returning the position of the option with the highest utility
    # that is available in the menu
    
end

Let's compute the likelihood funtion

We will assume $\epsilon \sim T1EV(0,1)$

In [66]:
function dcobj_t1ev(x,choice,dist,N,numSch,menu)
    δ = [x[1:numSch-1];0.0];
    γ = x[numSch];
    v = exp.(δ'.-γ*dist).*menu;
    PredProb = v ./ repeat(sum(v,dims=2),1,numSch);
    chocProb = [PredProb[i,choice[i]] for i in 1:N];
    objVal   =-sum(log, chocProb);
    
    return objVal
end

dcobj_t1ev (generic function with 1 method)

Let's call the optimizer

In [67]:
xtrue =     [δ[1:numSch-1].-δ[numSch];γ];
xinit = 0.9*[δ[1:numSch-1].-δ[numSch];γ];

Let's test the likelihood function first.

In [68]:
dcobj_t1ev(xinit,choice,dist,N,numSch,menu)

457.8313102634201

In [69]:
using Optim
using Optim: converged, maximum, maximizer, minimizer, iterations #some extra functions

In [70]:
results = optimize(x -> dcobj_t1ev(x,choice,dist,N,numSch, menu), xinit)

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [1.022683355031803,1.0992805399614345, ...]
 * Minimizer: [1.5078846889211288,1.5115568978947496, ...]
 * Minimum: 4.472344e+02
 * Iterations: 168
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 299

In [71]:
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in $(results.iterations) iterations")

minimum = 447.23443515255144 with argmin = [1.50788, 1.51156, 0.333182, 0.877001, 1.56654] in 168 iterations


In [72]:
comparison = (θhat = results.minimizer, θtrue = xtrue)

(θhat = [1.50788, 1.51156, 0.333182, 0.877001, 1.56654], θtrue = [1.13631, 1.22142, 0.241392, 0.828582, 1.2])

Now, lets assume that  $\epsilon \sim N(0,\pi^2 /6)$.

In this case we have to simulate the distribution in order to get the likelihood function.

In [73]:
function dcobj_normal(x,choice,dist,N,numSch, menu,seed)
    δ = [x[1:numSch-1];0.0];
    γ = x[numSch];
    σ = sqrt((pi^2)/6);
    
    # Draw the epsilon to simulate the choice probabilities
    Random.seed!(seed+4); 
    numDraws=numSch*20;
    ϵsim = σ*randn(numDraws,numSch, N);
    
    #Compute the simulated utilities and the choice for each draws
    # Note that here we need only the probability of the best option, the other
    # probabilities do not matter here
    simChoice = zeros(Int64,numDraws,1);
    simProb   = zeros(N,1);
    
    for i=1:N
        simUtil = repeat(δ',numDraws,1) - γ*repeat(dist[i,:]',numDraws,1) + ϵsim[:,:,i];
        for k=1:numDraws
            simChoice[k]=findall(simUtil[k,:].== findmax(simUtil[k,menu[i,:]])[1])[1];
        end

        simProb[i] = sum(simChoice.==choice[i])/numDraws;

    end
    
    objVal   =-sum(log, simProb);
    
    return objVal
end

dcobj_normal (generic function with 2 methods)

Let's test the new likelihood function.

In [74]:
dcobj_normal(xinit,choice,dist,N,numSch,menu,seed)

462.28047338647497

Calling the optmizer again

In [75]:
results2 = optimize(x -> dcobj_normal(x,choice,dist,N,numSch,menu,seed), xinit)

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [1.022683355031803,1.0992805399614345, ...]
 * Minimizer: [1.182000655963147,1.3110638280973923, ...]
 * Minimum: 4.570797e+02
 * Iterations: 114
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 277

In [76]:
comparison2 = (θhat = results2.minimizer, θtrue = xtrue)

(θhat = [1.182, 1.31106, 0.090613, 0.73456, 1.33146], θtrue = [1.13631, 1.22142, 0.241392, 0.828582, 1.2])

Comparing the results:

In [77]:
θtrue = xtrue;
θhat_t1ev = results.minimizer;
θhat_normal = results2.minimizer;
[θtrue θhat_t1ev θhat_normal]

5×3 Array{Float64,2}:
 1.13631   1.50788   1.182   
 1.22142   1.51156   1.31106 
 0.241392  0.333182  0.090613
 0.828582  0.877001  0.73456 
 1.2       1.56654   1.33146 

As we discussed in the beginning, both estimatives have noise. However, the estimative assuming T1EV is considerably faster than the one assuming normal unobservables. We would have to perform some Monte Carlo simulation to check the consistency